# Analiza czerniaka za pomocą fraktalnej sieci neuronowej

In [1]:
import os
import datetime

import numpy as np
from scipy.ndimage import measurements

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_addons as tfa
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Sprawdzamy dostępne urządzenie

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

Zapisujemy konfigurację do zmiennych.

In [3]:
IMAGE_SIZE = 224
BATCH_SIZE = 32

Tworzymy callbacki do zbierania danych o wydajności modelu do Tensorboard, zapisywania modelu w trakcie jego trenowania i zatrzymania trenowania modelu, jeśli nie ma poprawy w wynikach w ciągu 10 epok. 

In [4]:
log_dir = '../logs/fit/' + datetime.datetime.now().strftime('fractal_net')
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [5]:
checkpoint_path = 'checkpoints/fractal_net.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                        checkpoint_path,
                        monitor='val_loss',
                        verbose=1,
                        save_best_only=True,
                        save_weights_only=False,
                        save_freq='epoch',
                        mode='auto')

In [6]:
early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_loss", 
                                                       min_delta=0.01, 
                                                       patience=10, 
                                                       restore_best_weights=True)

Definiujemy warstwę, która będzie tworzyła obraz z fraktalnych cech podanych jej obrazów.

In [7]:
class Fractal2D(tf.keras.layers.Layer):
    PERCOLATION_THRESHOLD = 0.59275
    
    def __init__(self):
        super(Fractal2D, self).__init__(name='fractal_layer')
        self.kernel_size_range = (3, 41)

    def chessboard_distance(self, patched_inputs, central_pixels, kernel_size):
        return tf.cast(
            tf.math.less_equal(
                tf.math.reduce_max(
                    tf.math.abs(tf.math.subtract(patched_inputs, central_pixels)), 
                    axis=3), 
                kernel_size), 
            dtype=tf.int32)
    
    def euclidean_distance(self, patched_inputs, central_pixels, kernel_size):
        return tf.cast(
            tf.math.less_equal(
                tf.math.pow(
                    tf.math.reduce_sum(
                        tf.math.pow(
                            tf.math.subtract(patched_inputs, central_pixels), 
                            2), 
                        axis=3), 
                    0.5), 
                kernel_size), 
            dtype=tf.int32)
    
    def manhattan_distance(self, patched_inputs, central_pixels, kernel_size):
        return tf.cast(
            tf.math.less_equal(
                tf.math.reduce_sum(
                    tf.math.abs(tf.math.subtract(patched_inputs, central_pixels)), 
                    axis=3), 
                kernel_size), 
            dtype=tf.int32)
    
    def extract_binary_patches(self, inputs, kernel_size, distance_function):
        patched_inputs = tf.image.extract_patches(inputs,
                                                     sizes=(1, kernel_size, kernel_size, 1),
                                                     strides=(1, kernel_size, kernel_size, 1),
                                                     rates=(1, 1, 1, 1),
                                                     padding='SAME')
        _, rows, cols, _ = patched_inputs.shape
        patched_inputs = tf.reshape(patched_inputs, shape=(-1, kernel_size, kernel_size, 3))
        
        central_pixels = tf.image.resize_with_crop_or_pad(patched_inputs, 1, 1)
        
        return tf.reshape(distance_function(patched_inputs, central_pixels, kernel_size), 
                          shape=(-1, rows * cols, kernel_size, kernel_size))
    
    def calculate_probability_matrices(self, binary_inputs, kernel_size):
        number_of_ones = tf.map_fn(lambda binary_input: tf.map_fn(lambda binary_patch: tf.math.reduce_sum(binary_patch), 
                                                                  binary_input), 
                                   binary_inputs)
        _, patch_number = number_of_ones.shape
        return tf.math.bincount(number_of_ones,
                                minlength=1, 
                                maxlength=kernel_size ** 2, 
                                axis=-1) / patch_number
    
    def calculate_fractal_dimensions(self, probability_matrices):
        def fd_helper(matrix):
            return tf.math.reduce_sum(tf.math.divide(matrix, tf.range(1, len(matrix) + 1, dtype=tf.float64)))
        return tf.map_fn(lambda matrix: fd_helper(matrix), probability_matrices)
    
    def calculate_lacunarity(self, probability_matrices):
        def m_helper(matrix):
            return tf.math.reduce_sum(tf.math.multiply(matrix, tf.range(1, len(matrix) + 1, dtype=tf.float64)))
        
        def m2_helper(matrix):
            return tf.math.reduce_sum(tf.math.multiply(tf.math.pow(matrix, 2), tf.range(1, len(matrix) + 1, dtype=tf.float64)))
        
        return tf.map_fn(lambda probability_matrix: 
                         tf.math.divide(
                             tf.math.subtract(m2_helper(probability_matrix), 
                                               tf.math.pow(m_helper(probability_matrix), 2)), 
                             tf.math.pow(m_helper(probability_matrix), 2)), 
                         probability_matrices)
    
    def average_cluster_percolation(self, binary_inputs, kernel_size):
        number_of_ones = tf.map_fn(lambda binary_input: tf.map_fn(lambda binary_patch: tf.math.reduce_sum(binary_patch), 
                                                                  binary_input), 
                                   binary_inputs)
        
        return tf.math.reduce_mean(
                        tf.cast(
                            tf.math.greater_equal(
                                tf.math.divide(number_of_ones, kernel_size ** 2), 
                                self.PERCOLATION_THRESHOLD), 
                            dtype=tf.int32), 
                    axis=1)
    
    def average_cluster_number(self, binary_inputs):
        return tf.math.reduce_mean(
            tf.map_fn(
                lambda binary_input: tf.map_fn(
                    lambda patch: tf.math.reduce_max(tfa.image.connected_components(patch)), 
                    binary_input), 
                binary_inputs), 
            axis=1)
        
    def average_cluster_max_area(self, binary_inputs):    
        def most_common(array):
            _, _, counts = tf.unique_with_counts(array)
            return tf.math.reduce_max(counts)
        
        return tf.math.reduce_mean(
                tf.map_fn(lambda binary_input: 
                            tf.map_fn(lambda patch: 
                                        most_common(tf.reshape(tfa.image.connected_components(patch), shape=(-1,))), 
                                      binary_input), 
                          binary_inputs), axis=1)

    def calculate_components(self, inputs, kernel_size, distance_function):
        binary_patches = self.extract_binary_patches(inputs, kernel_size, distance_function)
        print(f'\t\t\t\tfractal2d: binary_patches shape = {binary_patches.shape}')

        probability_matrices = self.calculate_probability_matrices(binary_patches, kernel_size)
        print(f'\t\t\t\tfractal2d: probability_matrices shape = {probability_matrices.shape}')
        fractal_dimensions = self.calculate_fractal_dimensions(probability_matrices)
        print(f'\t\t\t\tfractal2d: fractal_dimensions shape = {fractal_dimensions.shape}')
        lacunarity = self.calculate_lacunarity(probability_matrices)
        print(f'\t\t\t\tfractal2d: lacunarity shape = {lacunarity.shape}')

        average_cluster_percolation = self.average_cluster_percolation(binary_patches, kernel_size)
        print(f'\t\t\t\tfractal2d: average_cluster_percolation shape = {average_cluster_percolation.shape}')
        average_cluster_number = self.average_cluster_number(binary_patches)
        print(f'\t\t\t\tfractal2d: average_cluster_number shape = {average_cluster_number.shape}')
        average_cluster_max_area = self.average_cluster_max_area(binary_patches)
        print(f'\t\t\t\tfractal2d: average_cluster_max_area shape = {average_cluster_max_area.shape}')

        return tf.convert_to_tensor((average_cluster_number,
                                    average_cluster_percolation,
                                    average_cluster_max_area,
                                    lacunarity,
                                    fractal_dimensions), dtype=tf.float64)
    
    def rearrage_metrics(self, components):
        def helper(components_input):
            length, = components_input.shape
            
            rearranged_components = tf.concat([
                tf.boolean_mask(components_input, tf.range(length) % 5 == 0),
                tf.boolean_mask(components_input, tf.range(length) % 5 == 1),
                tf.boolean_mask(components_input, tf.range(length) % 5 == 2),
                tf.boolean_mask(components_input, tf.range(length) % 5 == 3),
                tf.boolean_mask(components_input, tf.range(length) % 5 == 4),
            ], axis=0)
            return rearranged_components
        return tf.map_fn(helper, components)
    
    
    def call(self, inputs):
        kernel_size_start, kernel_size_end = self.kernel_size_range

        
        cd_components, ed_components, md_components = [], [], []
        for kernel_size in range(kernel_size_start, kernel_size_end + 1, 2):
            print(f'\t\tfractald2d: kernel_size = {kernel_size}')
            cd_components.append(
                tf.transpose(
                    self.calculate_components(inputs,
                                           kernel_size, 
                                           distance_function=self.chessboard_distance)))
            print(f'\t\tfractald2d: added CD')
            ed_components.append(
                tf.transpose(
                    self.calculate_components(inputs,
                                           kernel_size, 
                                           distance_function=self.euclidean_distance)))
            print(f'\t\tfractald2d: added ED')
            md_components.append(
                tf.transpose(
                    self.calculate_components(inputs,
                                           kernel_size, 
                                           distance_function=self.manhattan_distance)))
            print(f'\t\tfractald2d: added MD')
            
        cd_components = tf.reshape(self.rearrage_metrics(tf.concat(cd_components, axis=1)), shape=(-1, 10, 10))
        ed_components = tf.reshape(self.rearrage_metrics(tf.concat(ed_components, axis=1)), shape=(-1, 10, 10))
        md_components = tf.reshape(self.rearrage_metrics(tf.concat(md_components, axis=1)), shape=(-1, 10, 10))
        
        outputs = tf.concat([tf.expand_dims(cd_components, axis=3), 
                             tf.expand_dims(ed_components, axis=3),
                             tf.expand_dims(md_components, axis=3)], 
                            axis=3)
        
        return tf.image.resize(outputs, size=(224, 224))

Ładujemy dane do trenowania i walidacji.

In [8]:
datagen = ImageDataGenerator(validation_split=0.2, rescale=1.0)
training_set = datagen.flow_from_directory('/small-data',
                                           target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                           batch_size=BATCH_SIZE,
                                           class_mode='categorical',
                                           subset='training')
validation_set = datagen.flow_from_directory('/small-data',
                                             target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                             batch_size=BATCH_SIZE,
                                             class_mode='categorical',
                                             subset='validation')

Found 640 images belonging to 3 classes.
Found 159 images belonging to 3 classes.


Zapisujemy ilość rozpoznawalnych diagnoz.

In [9]:
DIAGNOSIS_NUMBER = len(training_set.class_indices)

Definiujemy model, który składa się z modelu konwolucyjnego i modelu z warstwą fraktalną.

In [10]:
class FractalModel(tf.keras.Model):
    def __init__(self, input_shape, class_number):
        super(FractalModel, self).__init__(self, name='fractal_model')
        
        self.input_shape_ = (None, ) + input_shape
        self.fractal2d = Fractal2D()
        self.mobilenet_v2 = hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4", 
                                           output_shape=[1280],
                                           trainable=False)
        self.score = tf.keras.layers.Dense(class_number, activation='softmax')
        
    def call(self, inputs):
        inputs = tf.ensure_shape(inputs, self.input_shape_)
        print(f'input layer: input = {inputs.shape}')
        
        fractal = inputs
        fractal = self.fractal2d(fractal)
        print(f'fractal: fractal2d = {fractal.shape}')
        fractal = self.mobilenet_v2(fractal)
        print(f'fractal: mobilenet_v2 = {fractal.shape}')
        
        ordinary = inputs
        ordinary = self.mobilenet_v2(ordinary)
        print(f'ordinary: mobilenet_v2 = {ordinary.shape}')
        
        combine = tf.math.add(fractal, ordinary)
        print(f'combine: tf.math.add = {combine.shape}')
        score = self.score(combine)
        print(f'score: score = {score.shape}')
        
        return score

Trenujemy powyższy model.

In [ ]:
fractal_model = FractalModel(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), class_number=DIAGNOSIS_NUMBER)
fractal_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
fractal_model.fit(training_set, validation_data=validation_set, epochs=200, callbacks=[tensorboard_callback,
                                                                               checkpoint_callback,
                                                                               early_stop_callback])

input layer: input = (32, 224, 224, 3)
		fractald2d: kernel_size = 3
				fractal2d: binary_patches shape = (32, 5625, 3, 3)
				fractal2d: probability_matrices shape = (32, 9)
				fractal2d: fractal_dimensions shape = (32,)
				fractal2d: lacunarity shape = (32,)
				fractal2d: average_cluster_percolation shape = (32,)
				fractal2d: average_cluster_number shape = (32,)
				fractal2d: average_cluster_max_area shape = (32,)
		fractald2d: added CD
				fractal2d: binary_patches shape = (32, 5625, 3, 3)
				fractal2d: probability_matrices shape = (32, 9)
				fractal2d: fractal_dimensions shape = (32,)
				fractal2d: lacunarity shape = (32,)
				fractal2d: average_cluster_percolation shape = (32,)
				fractal2d: average_cluster_number shape = (32,)
				fractal2d: average_cluster_max_area shape = (32,)
		fractald2d: added ED
				fractal2d: binary_patches shape = (32, 5625, 3, 3)
				fractal2d: probability_matrices shape = (32, 9)
				fractal2d: fractal_dimensions shape = (32,)
				fractal2d:

				fractal2d: average_cluster_max_area shape = (32,)
		fractald2d: added MD
		fractald2d: kernel_size = 17
				fractal2d: binary_patches shape = (32, 196, 17, 17)
				fractal2d: probability_matrices shape = (32, 289)
				fractal2d: fractal_dimensions shape = (32,)
				fractal2d: lacunarity shape = (32,)
				fractal2d: average_cluster_percolation shape = (32,)
				fractal2d: average_cluster_number shape = (32,)
				fractal2d: average_cluster_max_area shape = (32,)
		fractald2d: added CD
				fractal2d: binary_patches shape = (32, 196, 17, 17)
				fractal2d: probability_matrices shape = (32, 289)
				fractal2d: fractal_dimensions shape = (32,)
				fractal2d: lacunarity shape = (32,)
				fractal2d: average_cluster_percolation shape = (32,)
				fractal2d: average_cluster_number shape = (32,)
				fractal2d: average_cluster_max_area shape = (32,)
		fractald2d: added ED
				fractal2d: binary_patches shape = (32, 196, 17, 17)
				fractal2d: probability_matrices shape = (32, 289)
				fractal2d:

				fractal2d: average_cluster_number shape = (32,)
				fractal2d: average_cluster_max_area shape = (32,)
		fractald2d: added MD
		fractald2d: kernel_size = 31
				fractal2d: binary_patches shape = (32, 64, 31, 31)
				fractal2d: probability_matrices shape = (32, 961)
				fractal2d: fractal_dimensions shape = (32,)
				fractal2d: lacunarity shape = (32,)
				fractal2d: average_cluster_percolation shape = (32,)
				fractal2d: average_cluster_number shape = (32,)
				fractal2d: average_cluster_max_area shape = (32,)
		fractald2d: added CD
				fractal2d: binary_patches shape = (32, 64, 31, 31)
				fractal2d: probability_matrices shape = (32, 961)
				fractal2d: fractal_dimensions shape = (32,)
				fractal2d: lacunarity shape = (32,)
				fractal2d: average_cluster_percolation shape = (32,)
				fractal2d: average_cluster_number shape = (32,)
				fractal2d: average_cluster_max_area shape = (32,)
		fractald2d: added ED
				fractal2d: binary_patches shape = (32, 64, 31, 31)
				fractal2d: prob

				fractal2d: average_cluster_percolation shape = (None,)
				fractal2d: average_cluster_number shape = (None,)
				fractal2d: average_cluster_max_area shape = (None,)
		fractald2d: added MD
		fractald2d: kernel_size = 5
				fractal2d: binary_patches shape = (None, 2025, 5, 5)
				fractal2d: probability_matrices shape = <unknown>
				fractal2d: fractal_dimensions shape = (None,)
				fractal2d: lacunarity shape = (None,)
				fractal2d: average_cluster_percolation shape = (None,)
				fractal2d: average_cluster_number shape = (None,)
				fractal2d: average_cluster_max_area shape = (None,)
		fractald2d: added CD
				fractal2d: binary_patches shape = (None, 2025, 5, 5)
				fractal2d: probability_matrices shape = <unknown>
				fractal2d: fractal_dimensions shape = (None,)
				fractal2d: lacunarity shape = (None,)
				fractal2d: average_cluster_percolation shape = (None,)
				fractal2d: average_cluster_number shape = (None,)
				fractal2d: average_cluster_max_area shape = (None,)
		fractald2

				fractal2d: average_cluster_max_area shape = (None,)
		fractald2d: added MD
		fractald2d: kernel_size = 19
				fractal2d: binary_patches shape = (None, 144, 19, 19)
				fractal2d: probability_matrices shape = <unknown>
				fractal2d: fractal_dimensions shape = (None,)
				fractal2d: lacunarity shape = (None,)
				fractal2d: average_cluster_percolation shape = (None,)
				fractal2d: average_cluster_number shape = (None,)
				fractal2d: average_cluster_max_area shape = (None,)
		fractald2d: added CD
				fractal2d: binary_patches shape = (None, 144, 19, 19)
				fractal2d: probability_matrices shape = <unknown>
				fractal2d: fractal_dimensions shape = (None,)
				fractal2d: lacunarity shape = (None,)
				fractal2d: average_cluster_percolation shape = (None,)
				fractal2d: average_cluster_number shape = (None,)
				fractal2d: average_cluster_max_area shape = (None,)
		fractald2d: added ED
				fractal2d: binary_patches shape = (None, 144, 19, 19)
				fractal2d: probability_matrices shap

				fractal2d: binary_patches shape = (None, 49, 33, 33)
				fractal2d: probability_matrices shape = <unknown>
				fractal2d: fractal_dimensions shape = (None,)
				fractal2d: lacunarity shape = (None,)
				fractal2d: average_cluster_percolation shape = (None,)
				fractal2d: average_cluster_number shape = (None,)
				fractal2d: average_cluster_max_area shape = (None,)
		fractald2d: added CD
				fractal2d: binary_patches shape = (None, 49, 33, 33)
				fractal2d: probability_matrices shape = <unknown>
				fractal2d: fractal_dimensions shape = (None,)
				fractal2d: lacunarity shape = (None,)
				fractal2d: average_cluster_percolation shape = (None,)
				fractal2d: average_cluster_number shape = (None,)
				fractal2d: average_cluster_max_area shape = (None,)
		fractald2d: added ED
				fractal2d: binary_patches shape = (None, 49, 33, 33)
				fractal2d: probability_matrices shape = <unknown>
				fractal2d: fractal_dimensions shape = (None,)
				fractal2d: lacunarity shape = (None,)
				fra

				fractal2d: average_cluster_number shape = (None,)
				fractal2d: average_cluster_max_area shape = (None,)
		fractald2d: added MD
		fractald2d: kernel_size = 7
				fractal2d: binary_patches shape = (None, 1024, 7, 7)
				fractal2d: probability_matrices shape = <unknown>
				fractal2d: fractal_dimensions shape = (None,)
				fractal2d: lacunarity shape = (None,)
				fractal2d: average_cluster_percolation shape = (None,)
				fractal2d: average_cluster_number shape = (None,)
				fractal2d: average_cluster_max_area shape = (None,)
		fractald2d: added CD
				fractal2d: binary_patches shape = (None, 1024, 7, 7)
				fractal2d: probability_matrices shape = <unknown>
				fractal2d: fractal_dimensions shape = (None,)
				fractal2d: lacunarity shape = (None,)
				fractal2d: average_cluster_percolation shape = (None,)
				fractal2d: average_cluster_number shape = (None,)
				fractal2d: average_cluster_max_area shape = (None,)
		fractald2d: added ED
				fractal2d: binary_patches shape = (None, 10

				fractal2d: average_cluster_percolation shape = (None,)
				fractal2d: average_cluster_number shape = (None,)
				fractal2d: average_cluster_max_area shape = (None,)
		fractald2d: added MD
		fractald2d: kernel_size = 21
				fractal2d: binary_patches shape = (None, 121, 21, 21)
				fractal2d: probability_matrices shape = <unknown>
				fractal2d: fractal_dimensions shape = (None,)
				fractal2d: lacunarity shape = (None,)
				fractal2d: average_cluster_percolation shape = (None,)
				fractal2d: average_cluster_number shape = (None,)
				fractal2d: average_cluster_max_area shape = (None,)
		fractald2d: added CD
				fractal2d: binary_patches shape = (None, 121, 21, 21)
				fractal2d: probability_matrices shape = <unknown>
				fractal2d: fractal_dimensions shape = (None,)
				fractal2d: lacunarity shape = (None,)
				fractal2d: average_cluster_percolation shape = (None,)
				fractal2d: average_cluster_number shape = (None,)
				fractal2d: average_cluster_max_area shape = (None,)
		fracta

				fractal2d: average_cluster_number shape = (None,)
				fractal2d: average_cluster_max_area shape = (None,)
		fractald2d: added MD
		fractald2d: kernel_size = 35
				fractal2d: binary_patches shape = (None, 49, 35, 35)
				fractal2d: probability_matrices shape = <unknown>
				fractal2d: fractal_dimensions shape = (None,)
				fractal2d: lacunarity shape = (None,)
				fractal2d: average_cluster_percolation shape = (None,)
				fractal2d: average_cluster_number shape = (None,)
				fractal2d: average_cluster_max_area shape = (None,)
		fractald2d: added CD
				fractal2d: binary_patches shape = (None, 49, 35, 35)
				fractal2d: probability_matrices shape = <unknown>
				fractal2d: fractal_dimensions shape = (None,)
				fractal2d: lacunarity shape = (None,)
				fractal2d: average_cluster_percolation shape = (None,)
				fractal2d: average_cluster_number shape = (None,)
				fractal2d: average_cluster_max_area shape = (None,)
		fractald2d: added ED
				fractal2d: binary_patches shape = (None, 4